<a href="https://colab.research.google.com/github/zrruziev/Deep_Learning_Practice/blob/main/TDC/C3_W1_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Download the BBC_News_Classification_Dataset

In [55]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv 

--2022-04-06 08:41:45--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.128, 173.194.202.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘bbc-text.csv’

bbc-text.csv        100%[===================>]   4.82M  --.-KB/s    in 0.02s   

2022-04-06 08:41:45 (195 MB/s) - ‘bbc-text.csv’ saved [5057493/5057493]



##Import Dependencies

In [56]:
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

##Show the header of the csv file and some of its data


In [57]:
with open('bbc-text.csv', 'r') as csvfile:
    print(f'First Line (header) of the data: \n\n{csvfile.readline()}')
    print(f'Each data points look like this: \n\n{csvfile.readline()}')

First Line (header) of the data: 

category,text

Each data points look like this: 

tech,tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  

##Read the csvfile and remove stopwords

In [58]:
def remove_stopwords(file):
    stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    sentences  = []    
    labels = []
    with open(file) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            labels.append(row[0])
            sentence = row[1].lower()
            for word in stopwords:
                token = " " + word + " "
                sentence = sentence.replace(token, " ")
            sentences.append(sentence)
    return sentences, labels

##Make sentences and labels Ready!

In [59]:
(sentences, labels) = remove_stopwords('bbc-text.csv')

In [60]:
print(f'There are {len(sentences)} sentences in the dataset.')
print(f'\nFirst sentence has {len(sentences[0].split())} words (after removing stopwords.)')
print(f'\nThere are {len(labels)} labels in the dataset.')
print(f'\nThe first 5 labels are {labels[:5]}')

There are 2225 sentences in the dataset.

First sentence has 436 words (after removing stopwords.)

There are 2225 labels in the dataset.

The first 5 labels are ['tech', 'business', 'sport', 'sport', 'entertainment']


##Using Tokenizer for sentences

In [61]:
def fit_tokenizer(sentences):
    tokenizer = Tokenizer(oov_token='<OOV>')
    # tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    return tokenizer

In [62]:
tokenizer = fit_tokenizer(sentences)
word_index = tokenizer.word_index

print(f'Vocabulary contains {len(word_index)} words.\n')
print(f'<OOV> token included in vocabulary' if '<OOV>' in word_index else '<OOV> token NOT included in vocabulary')

Vocabulary contains 29714 words.

<OOV> token included in vocabulary


In [63]:
def get_padded_sequences(tokenizer, sentences):
    sequences = tokenizer.texts_to_sequences(sentences)
    padded_sequences = pad_sequences(sequences, padding='post')
    return padded_sequences



In [64]:
padded_sequences = get_padded_sequences(tokenizer, sentences)

print(f'First padded sequence looks like this: \n\n{padded_sequences[0]}\n')
print(f'Numpy array of all sequences has shape: {padded_sequences.shape}\n')
print(f'This means there are {padded_sequences.shape[0]} sequences in total and each one has a size of {padded_sequences.shape[1]}')

First padded sequence looks like this: 

[  96  176 1158 ...    0    0    0]

Numpy array of all sequences has shape: (2225, 2441)

This means there are 2225 sequences in total and each one has a size of 2441


##Using Tokenizer for labels

In [65]:
def tokenize_labels(labels):
    label_tokenizer = Tokenizer()
    label_tokenizer.fit_on_texts(labels)
    label_word_index = label_tokenizer.word_index
    label_sequences = label_tokenizer.texts_to_sequences(labels)

    return label_sequences, label_word_index

In [66]:
label_sequences, label_word_index = tokenize_labels(labels)

print(f'Vocabulary of labels loks like this: {label_word_index}\n')
print(f'First ten sequences {label_sequences[:10]}')

Vocabulary of labels loks like this: {'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}

First ten sequences [[4], [2], [1], [1], [5], [3], [3], [1], [1], [5]]
